## Zadanie 3

In [0]:
from pyspark.sql.functions import *
from pyspark.sql import Window

In [0]:
# Dane dla Transactions
transactions_data = [
( 1, '2011-01-01', 500),
( 1, '2011-01-15', 50),
( 1, '2011-01-22', 250),
( 1, '2011-01-24', 75),
( 1, '2011-01-26', 125),
( 1, '2011-01-28', 175),
( 2, '2011-01-01', 500),
( 2, '2011-01-15', 50),
( 2, '2011-01-22', 25),
( 2, '2011-01-23', 125),
( 2, '2011-01-26', 200),
( 2, '2011-01-29', 250),
( 3, '2011-01-01', 500),
( 3, '2011-01-15', 50 ),
( 3, '2011-01-22', 5000),
( 3, '2011-01-25', 550),
( 3, '2011-01-27', 95 ),
( 3, '2011-01-30', 2500)
]

transactions_df = spark.createDataFrame(transactions_data, ["AccountId", "TranDate", "TranAmt"])
transactions_df = transactions_df.withColumn("TranDate", to_date("TranDate", "yyyy-MM-dd"))
display(transactions_df)

# Dane dla Logical
logical_data = [
(1,'George', 800),
(2,'Sam', 950),
(3,'Diane', 1100),
(4,'Nicholas', 1250),
(5,'Samuel', 1250),
(6,'Patricia', 1300),
(7,'Brian', 1500),
(8,'Thomas', 1600),
(9,'Fran', 2450),
(10,'Debbie', 2850),
(11,'Mark', 2975),
(12,'James', 3000),
(13,'Cynthia', 3000),
(14,'Christopher', 5000)
]

logical_df = spark.createDataFrame(logical_data, ["RowID", "FName", "Salary"])
display(logical_df)

AccountId,TranDate,TranAmt
1,2011-01-01,500
1,2011-01-15,50
1,2011-01-22,250
1,2011-01-24,75
1,2011-01-26,125
1,2011-01-28,175
2,2011-01-01,500
2,2011-01-15,50
2,2011-01-22,25
2,2011-01-23,125


RowID,FName,Salary
1,George,800
2,Sam,950
3,Diane,1100
4,Nicholas,1250
5,Samuel,1250
6,Patricia,1300
7,Brian,1500
8,Thomas,1600
9,Fran,2450
10,Debbie,2850


In [0]:
window = Window.partitionBy("AccountId").orderBy("TranDate")
display(transactions_df.withColumn("RunTotalAmt", sum("TranAmt").over(window)).orderBy(["AccountId","TranDate"]))

AccountId,TranDate,TranAmt,RunTotalAmt
1,2011-01-01,500,500
1,2011-01-15,50,550
1,2011-01-22,250,800
1,2011-01-24,75,875
1,2011-01-26,125,1000
1,2011-01-28,175,1175
2,2011-01-01,500,500
2,2011-01-15,50,550
2,2011-01-22,25,575
2,2011-01-23,125,700


In [0]:
display(transactions_df.withColumn("RunAvg", avg("TranAmt").over(window))\
    .withColumn("RunTranQty", count("*").over(window))\
    .withColumn("RunMinAmt", min("TranAmt").over(window))\
    .withColumn("RunMaxAmt", max("TranAmt").over(window))\
    .withColumn("RunTotalAmt", sum("TranAmt").over(window))\
    .orderBy(["AccountId","TranDate"]))

AccountId,TranDate,TranAmt,RunAvg,RunTranQty,RunMinAmt,RunMaxAmt,RunTotalAmt
1,2011-01-01,500,500.0,1,500,500,500
1,2011-01-15,50,275.0,2,50,500,550
1,2011-01-22,250,266.6666666666667,3,50,500,800
1,2011-01-24,75,218.75,4,50,500,875
1,2011-01-26,125,200.0,5,50,500,1000
1,2011-01-28,175,195.83333333333334,6,50,500,1175
2,2011-01-01,500,500.0,1,500,500,500
2,2011-01-15,50,275.0,2,50,500,550
2,2011-01-22,25,191.66666666666666,3,25,500,575
2,2011-01-23,125,175.0,4,25,500,700


In [0]:
sliding_window = Window.partitionBy("AccountId").orderBy("TranDate").rowsBetween(-2, 0)
sliding_window_2 = Window.partitionBy("AccountId").orderBy("TranDate")

display(transactions_df.withColumn("SlideAvg", avg("TranAmt").over(sliding_window))\
    .withColumn("SlideQty", count("*").over(sliding_window))\
    .withColumn("SlideMin", min("TranAmt").over(sliding_window))\
    .withColumn("SlideMax", max("TranAmt").over(sliding_window))\
    .withColumn("SlideTotal", sum("TranAmt").over(sliding_window))\
    .withColumn("RowNumber", row_number().over(sliding_window_2))\
    .orderBy(["AccountId","TranDate", "RowNumber"]))


AccountId,TranDate,TranAmt,SlideAvg,SlideQty,SlideMin,SlideMax,SlideTotal,RowNumber
1,2011-01-01,500,500.0,1,500,500,500,1
1,2011-01-15,50,275.0,2,50,500,550,2
1,2011-01-22,250,266.6666666666667,3,50,500,800,3
1,2011-01-24,75,125.0,3,50,250,375,4
1,2011-01-26,125,150.0,3,75,250,450,5
1,2011-01-28,175,125.0,3,75,175,375,6
2,2011-01-01,500,500.0,1,500,500,500,1
2,2011-01-15,50,275.0,2,50,500,550,2
2,2011-01-22,25,191.66666666666666,3,25,500,575,3
2,2011-01-23,125,66.66666666666667,3,25,125,200,4


In [0]:
window_rows_unbounded = Window.orderBy("Salary").rowsBetween(Window.unboundedPreceding, Window.currentRow)
window_range_unbounded = Window.orderBy("Salary").rangeBetween(Window.unboundedPreceding, Window.currentRow)

display(logical_df.withColumn("SumByRows", sum("Salary").over(window_rows_unbounded))\
    .withColumn("SumByRange", sum("Salary").over(window_range_unbounded))\
    .orderBy("RowID"))

RowID,FName,Salary,SumByRows,SumByRange
1,George,800,800,800
2,Sam,950,1750,1750
3,Diane,1100,2850,2850
4,Nicholas,1250,4100,5350
5,Samuel,1250,5350,5350
6,Patricia,1300,6650,6650
7,Brian,1500,8150,8150
8,Thomas,1600,9750,9750
9,Fran,2450,12200,12200
10,Debbie,2850,15050,15050
